In [27]:
import pandas as pd
import plotly.graph_objects as go
import re

# Soppressione degli avvertimenti
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def plot_top_movies_trends(selected_ranks, start_year, end_year):
    top_movies_data = []

    for year in range(start_year, end_year + 1):
        file_path = f'../csv_guadagni/ranking_summary_{year}.csv'

        try:
            df_data = pd.read_csv(file_path)
            df_data['worldwide'] = df_data['worldwide'].replace('[\$,]', '', regex=True).astype(float)
            df_data = df_data.sort_values(by='worldwide', ascending=False)

            match = re.search(r'_(\d{4})\.csv', file_path)
            if match:
                plot_year = int(match.group(1))
            else:
                plot_year = None

            for index, row in df_data.head(5).iterrows():
                if row['rank'] in selected_ranks:
                    top_movies_data.append({'Year': plot_year, 'Earnings': row['worldwide'],
                                            'Title': row['title'], 'Rank': row['rank']})
        except FileNotFoundError:
            print(f"File not found for year {year}")
            continue

    top_movies_df = pd.DataFrame(top_movies_data)
    top_movies_df = top_movies_df.sort_values(by=['Year', 'Rank'])

    # Definisci una lista di colori pastello personalizzati
    pastel_colors = ['#8bb3c4', '#93bed0', '#9cc9dc', '#a4d4e8', '#addff4']
    rank_labels = {1: 'Rank 1', 2: 'Rank 2', 3: 'Rank 3', 4: 'Rank 4', 5: 'Rank 5'}

    fig = go.Figure()

    for rank, color in zip(selected_ranks, pastel_colors):
        filtered_df = top_movies_df[top_movies_df['Rank'] == rank]
        fig.add_trace(go.Bar(
            x=filtered_df['Year'],
            y=filtered_df['Earnings'],
            name=rank_labels[rank],
            marker_color=color,
            hovertext=filtered_df['Title'],
            hoverinfo='text+y'  # Mostra solo il titolo del film e il guadagno nell'hover
        ))

    fig.update_layout(
        barmode='group',
        title='Andamento del Guadagno dei Top 5 film nel corso degli anni',
        xaxis_title='Anno',
        yaxis_title='Guadagno al botteghino (Dollari)',
        template='plotly_white',
        height=800,
        width=1500,
        showlegend=False
    )

    fig.write_html("../file_html/top_movies_trends3.html")
    fig.show()

# Seleziona i rank da mostrare e il range degli anni
plot_top_movies_trends([1, 2, 3, 4, 5], 2020, 2024)
